In [ ]:
! pip install tabpfn #pip install "tabpfn @ git+https://github.com/PriorLabs/TabPFN.git"

# Setting Up Data Science Environment for Classification

This code sets up a Python environment for a classification task, specifically preparing to work with the breast cancer dataset. Here's what each import provides:


### Machine Learning Framework
- `load_breast_cancer`: A function to load the breast cancer Wisconsin dataset, a common benchmark for binary classification
  - Features are computed from a digitized image of a fine needle aspirate of a breast mass
  - The dataset includes 569 samples with 30 features
  - The target variable indicates whether a tumor is malignant or benign

- `accuracy_score` and `roc_auc_score`: Metrics for evaluating classification models
  - Accuracy: The proportion of correct predictions
  - ROC-AUC: Area under the Receiver Operating Characteristic curve, measuring the model's ability to discriminate between classes

- `train_test_split`: A function to split datasets into random train and test subsets for model validation

This code prepares the environment for building and evaluating a classification model, likely using TabPFN (based on your previous pip install) as the modeling approach.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix, 
    classification_report, 
    precision_score, 
    recall_score, 
    f1_score, 
    roc_curve, 
    roc_auc_score
)
from sklearn.model_selection import train_test_split

# Load the breast cancer dataset

In [3]:

X, y = load_breast_cancer(return_X_y=True)

# Get the feature names and target names

In [4]:

cancer = load_breast_cancer()
feature_names = cancer.feature_names
target_names = cancer.target_names


# Create a DataFrame for easier exploration

In [ ]:

df = pd.DataFrame(X, columns=feature_names)
df['target'] = y
df['diagnosis'] = [target_names[val] for val in y]


# Basic dataset information

In [ ]:

print("Dataset Shape:", X.shape)
print("Number of Features:", X.shape[1])
print("Number of Samples:", X.shape[0])
print("\nClass Distribution:")
print(df['diagnosis'].value_counts())
print("\nClass Distribution (%):")
print(df['diagnosis'].value_counts(normalize=True).round(3) * 100)


In [ ]:
# Display first few rows of the dataset
print("\nFirst 5 rows of the dataset:")
print(df.head())

# Statistical summary of features
print("\nStatistical Summary of Features:")
print(df.describe().T)

# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum().sum())



# Visualize the correlation matrix

In [ ]:

plt.figure(figsize=(16, 14))
correlation_matrix = df.drop('diagnosis', axis=1).corr()
mask = np.triu(correlation_matrix)
sns.heatmap(correlation_matrix, annot=False, mask=mask, cmap='coolwarm', linewidths=0.5)
plt.title('Feature Correlation Matrix', fontsize=15)
plt.tight_layout()
plt.show()

# Boxplots for comparing feature distributions between classes
plt.figure(figsize=(15, 10))
for i, feature in enumerate(feature_names[:5]):  # Just the first 5 features for clarity
    plt.subplot(2, 3, i+1)
    sns.boxplot(x='diagnosis', y=feature, data=df)
    plt.title(f'{feature} by Diagnosis')
plt.tight_layout()
plt.show()

# Distribution of a few key features
plt.figure(figsize=(15, 10))
for i, feature in enumerate(feature_names[:5]):  # Just the first 5 features
    plt.subplot(2, 3, i+1)
    sns.histplot(df[feature], kde=True, bins=30)
    plt.title(f'Distribution of {feature}')
plt.tight_layout()
plt.show()


# Boxplots for comparing feature distributions between classes

In [ ]:

plt.figure(figsize=(15, 10))
for i, feature in enumerate(feature_names[:5]):  # Just the first 5 features for clarity
    plt.subplot(2, 3, i+1)
    sns.boxplot(x='diagnosis', y=feature, data=df)
    plt.title(f'{feature} by Diagnosis')
plt.tight_layout()
plt.show()



# Distribution of a few key features

In [ ]:
plt.figure(figsize=(15, 10))
for i, feature in enumerate(feature_names[:5]):  # Just the first 5 features
    plt.subplot(2, 3, i+1)
    sns.histplot(df[feature], kde=True, bins=30)
    plt.title(f'Distribution of {feature}')
plt.tight_layout()
plt.show()


# Using TabPFN for Breast Cancer Classification

This code imports the TabPFN classifier and loads the breast cancer dataset for a classification task:

### TabPFN Import
- `TabPFNClassifier`: A specialized classifier from the TabPFN library that uses Prior-Data Fitted Networks
  - TabPFN is designed specifically for tabular data
  - It leverages transformer-based architectures pre-trained on synthetic tabular datasets
  - The model requires minimal hyperparameter tuning and often performs well out-of-the-box

### Dataset Loading
- `load_breast_cancer(return_X_y=True)`: Loads the breast cancer Wisconsin dataset as NumPy arrays
  - `X`: Features matrix with 569 samples and 30 features (measurements from digitized images)
  - `y`: Target vector with binary values indicating benign (0) or malignant (1) tumors
  - The `return_X_y=True` parameter specifies that the function should return the data and target as separate arrays rather than as a Bunch object

This setup forms the foundation for applying the TabPFN classifier to predict breast cancer diagnoses based on the provided features. TabPFN is particularly well-suited for this type of tabular classification task with a moderate number of features.

In [ ]:
from tabpfn import TabPFNClassifier

# Load data
X, y = load_breast_cancer(return_X_y=True)


# Splitting Data for Classification Model Evaluation


This code divides the breast cancer dataset into training and testing sets using scikit-learn's `train_test_split` function:

### Parameters Explained:
- `X`: The feature matrix containing 30 features for each sample
- `y`: The target vector with binary classification labels (0 for benign, 1 for malignant)
- `test_size=0.5`: Allocates 50% of the data for testing and 50% for training
  - This is a larger test set than the typical 20-30%, which might be chosen to better evaluate model performance
- `random_state=42`: Sets a specific random seed for reproducibility
  - Using a fixed random state ensures that the split will be the same each time the code runs
  - The value 42 is commonly used as a default in data science (a reference to "The Hitchhiker's Guide to the Galaxy")

### Result:
- `X_train`: Features for training the model (569 × 0.5 = ~284 samples)
- `X_test`: Features for evaluating the model (~284 samples)
- `y_train`: Target labels for the training set
- `y_test`: Target labels for the test set

This even split allows for robust evaluation of the TabPFN classifier's performance on unseen data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)



# Training the TabPFN Classifier

This code initializes and trains a TabPFN classifier on the breast cancer dataset:

### Model Initialization
- `TabPFNClassifier()`: Creates an instance of the TabPFN classifier with default parameters
  - TabPFN is a "Prior-Data Fitted Network" specifically designed for tabular data
  - Unlike traditional ML models, TabPFN comes pre-trained on synthetic tabular data
  - The default settings typically work well without extensive hyperparameter tuning

### Model Training
- `clf.fit(X_train, y_train)`: Trains the classifier on the training data
  - `X_train`: Feature matrix with ~284 samples and 30 features
  - `y_train`: Target vector with binary labels (benign/malignant)
  - TabPFN's training process is generally faster than traditional ML models as it leverages transfer learning
  - The model adapts its pre-trained knowledge to the specific patterns in the breast cancer dataset

TabPFN typically requires less data for effective training compared to models that start from scratch, making it well-suited for medical datasets like this one where labeled data might be limited.

In [ ]:
# Initialize a classifier
clf = TabPFNClassifier()
clf.fit(X_train, y_train)



# Making Predictions and Evaluating Performance

This code evaluates the TabPFN classifier by generating predictions and calculating the ROC AUC score:

### Prediction Generation
- `clf.predict_proba(X_test)`: Generates probability predictions for each class
  - Returns a 2D array of shape (n_samples, n_classes) = (~284 samples, 2 classes)
  - Each row represents one test sample
  - Each column represents the probability of belonging to a class (0 or 1)

### Performance Evaluation
- `prediction_probabilities[:, 1]`: Extracts only the probabilities for the positive class (malignant)
  - The second column (index 1) contains probabilities for the positive class

- `roc_auc_score(y_test, prediction_probabilities[:, 1])`: Calculates the ROC AUC metric
  - ROC (Receiver Operating Characteristic) curve plots the true positive rate against the false positive rate
  - AUC (Area Under Curve) measures the classifier's ability to distinguish between classes
  - Values range from 0 to 1, where:
    - 1.0 represents a perfect classifier
    - 0.5 represents a random classifier
    - Values above 0.8 are generally considered good
    - Values above 0.9 are considered excellent

This evaluation helps assess how well the TabPFN model distinguishes between benign and malignant tumors based on the provided features, which is crucial for medical diagnostic applications.

In [ ]:
# Predict probabilities
prediction_probabilities = clf.predict_proba(X_test)
print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))



# Predicting Class Labels and Evaluating Accuracy


This code produces class predictions and evaluates the model's accuracy:

### Prediction Generation
- `clf.predict(X_test)`: Generates class label predictions for the test data
  - Unlike `predict_proba()` which returns probabilities, this method returns the actual predicted class labels
  - Returns a 1D array of shape (n_samples,) with values 0 (benign) or 1 (malignant)
  - Internally, the model assigns the class with the highest probability to each sample

### Performance Evaluation
- `accuracy_score(y_test, predictions)`: Calculates the proportion of correct predictions
  - Compares the predicted labels (`predictions`) with the true labels (`y_test`)
  - Returns a value between 0 and 1, where:
    - 1.0 means all predictions are correct
    - 0.0 means all predictions are incorrect
  - Formula: (Number of correct predictions) / (Total number of predictions)

Accuracy is a straightforward metric for classification tasks, representing the percentage of samples correctly classified. For breast cancer diagnosis, high accuracy is important, but the ROC AUC score (calculated in the previous step) provides additional insight into the model's ability to distinguish between the classes, which is particularly valuable in medical contexts where false negatives can have serious consequences.

In [ ]:
# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

# Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Benign', 'Malignant'], 
            yticklabels=['Benign', 'Malignant'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# 3. Classification Report, Individual Metrics and RoC Curve

In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, predictions, 
                           target_names=['Benign', 'Malignant']))


print("\nKey Metrics:")
print(f"Accuracy:  {accuracy_score(y_test, predictions):.4f}")
print(f"Precision: {precision_score(y_test, predictions):.4f}")
print(f"Recall:    {recall_score(y_test, predictions):.4f}")
print(f"F1 Score:  {f1_score(y_test, predictions):.4f}")
print(f"ROC AUC:   {roc_auc_score(y_test, prediction_probabilities[:, 1]):.4f}")


fpr, tpr, _ = roc_curve(y_test, prediction_probabilities[:, 1])
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC (AUC = {roc_auc_score(y_test, prediction_probabilities[:, 1]):.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Installing and Using AutoTabPFN

This code enhances the TabPFN implementation with automatic hyperparameter optimization:

### Package Installation
- `hyperopt`: A Python library for serial and parallel optimization over awkward search spaces
- `tabpfn-extensions`: Extended functionality for TabPFN, including automated optimization tools
- The commented portion shows how you could alternatively clone the repository directly

### AutoTabPFN Setup
- `AutoTabPFNClassifier`: An enhanced version of TabPFN that performs automatic tuning and ensemble creation
  - Combines multiple TabPFN models with optimized configurations
  - Uses Bayesian optimization via hyperopt to find optimal hyperparameters

### Configuration Parameters
- `max_time=120`: Limits the tuning process to 120 seconds (2 minutes)
  - The optimizer will try to find the best configuration within this time constraint
  - Longer times generally lead to better results but with diminishing returns
- `device="cpu"`: Specifies that computations should run on the CPU
  - Alternative would be "cuda" for GPU acceleration if available

### Training Process
- `clf.fit(X_train, y_train)`: Trains the optimized ensemble model
  - During this process, AutoTabPFN will:
    1. Try different TabPFN configurations
    2. Evaluate their performance
    3. Select the best performing models
    4. Create an ensemble of these models

AutoTabPFN typically provides better performance than the base TabPFN model by leveraging ensemble techniques and hyperparameter optimization, though at the cost of longer training times.

In [ ]:
! pip install hyperopt tabpfn-extensions #git clone https://github.com/priorlabs/tabpfn-extensions.git

from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNClassifier

clf = AutoTabPFNClassifier(max_time=120, device="cpu") # 120 seconds tuning time
clf.fit(X_train, y_train)



In [ ]:
predictions = clf.predict(X_test)
print(predictions)

# Saving and Loading the TabPFN Model

Here's the code to save your trained TabPFN model and then load it back:


1. **Saving the Model**:
   - Uses Python's `pickle` module to serialize the model object
   - The `'wb'` flag opens the file in binary write mode
   - All model parameters and state are preserved in the file

2. **Loading the Model**:
   - Uses `pickle.load()` to deserialize the model from the file
   - The `'rb'` flag opens the file in binary read mode
   - The loaded model is stored in `loaded_clf`

3. **Important Considerations**:
   - Make sure the same version of TabPFN is used when loading the model
   - The pickle format is Python-specific and not cross-language compatible
   - For larger models, consider using joblib instead of pickle for better handling of large NumPy arrays

This approach works well for TabPFN models and allows you to reuse your trained model without having to retrain it each time.

In [ ]:
import pickle
import os

# After training your model with clf.fit(X_train, y_train)

# 1. Save the trained model to a file
model_filename = 'tabpfn_breast_cancer_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(clf, file)
print(f"Model saved to {model_filename}")

# 2. Later, to load the model from the file
if os.path.exists(model_filename):
    with open(model_filename, 'rb') as file:
        loaded_clf = pickle.load(file)
    print(f"Model loaded from {model_filename}")
    
    # Verify the loaded model works correctly
    test_predictions = loaded_clf.predict(X_test)
    test_accuracy = accuracy_score(y_test, test_predictions)
    print(f"Loaded model accuracy: {test_accuracy:.4f}")
else:
    print(f"Model file {model_filename} not found")

1. **Loads the saved model** using pickle
2. **Handles both single and multiple samples** by checking the input shape
3. **Returns a dictionary with comprehensive prediction information**:
   - Class labels (0/1)
   - Full probability matrix for all classes
   - Probability for the positive class only (simplified access)
4. **Includes error handling** for common issues like missing files or data format problems
5. **Provides example usage** for both single and batch prediction scenarios

You can easily incorporate this function into any application that needs to use your trained TabPFN model for breast cancer prediction.

In [ ]:
def predict_with_tabpfn(input_data, model_path='tabpfn_breast_cancer_model.pkl'):
   """
   Load a saved TabPFN model and make predictions for given input data.
   
   Parameters:
   -----------
   input_data : array-like
       Input features for prediction, shape should match training data.
       Can be a single sample (1D array) or multiple samples (2D array).
   model_path : str, default='tabpfn_breast_cancer_model.pkl'
       Path to the saved model file.
   
   Returns:
   --------
   dict: A dictionary containing:
       - 'predictions': Class labels (0/1)
       - 'probabilities': Probability estimates for each class
       - 'positive_proba': Probability of the positive class (class 1)
   
   Raises:
   -------
   FileNotFoundError: If the model file doesn't exist
   ValueError: If there's an issue with the input data format
   """
   import pickle
   import numpy as np
   import os
   
   # Check if model file exists
   if not os.path.exists(model_path):
       raise FileNotFoundError(f"Model file not found: {model_path}")
   
   # Load the model
   try:
       with open(model_path, 'rb') as file:
           model = pickle.load(file)
       print(f"Model successfully loaded from {model_path}")
   except Exception as e:
       raise Exception(f"Error loading model: {str(e)}")
   
   # Ensure input is in the right format
   input_data = np.asarray(input_data)
   if input_data.ndim == 1:
       # Single sample - reshape to 2D array
       input_data = input_data.reshape(1, -1)
   
   # Make predictions
   try:
       predictions = model.predict(input_data)
       probabilities = model.predict_proba(input_data)
       positive_class_proba = probabilities[:, 1]  # Probability of class 1 (usually positive class)
       
       return {
           'predictions': predictions,
           'probabilities': probabilities,
           'positive_proba': positive_class_proba
       }
   except Exception as e:
       raise ValueError(f"Error during prediction: {str(e)}")


# Example usage:
if __name__ == "__main__":
   # Example for a single sample
   # Replace with your actual test data
   sample = X_test[0]  # Single test instance
   
   result = predict_with_tabpfn(sample)
   print(f"Prediction: {'Malignant' if result['predictions'][0] == 1 else 'Benign'}")
   print(f"Confidence: {result['positive_proba'][0]:.4f}")
   
   # Example for multiple samples
   samples = X_test[:5]  # First 5 test instances
   
   batch_result = predict_with_tabpfn(samples)
   for i, (pred, prob) in enumerate(zip(batch_result['predictions'], batch_result['positive_proba'])):
       print(f"Sample {i+1}: {'Malignant' if pred == 1 else 'Benign'} (Confidence: {prob:.4f})")